# Spotify API


In [1]:
import os
import pandas as pd
import numpy as np
import json
import matplotlib.pyplot as plt
import seaborn as sns
%config InlineBackend.figure_format ='retina'
import spotipy
import spotipy.util as util
from spotipy.oauth2 import SpotifyClientCredentials, SpotifyOAuth
from spotipy import oauth2
import random
from functools import reduce
import requests
from spotify.spotify_creds import SPOTIFY_CLIENT_ID, SPOTIFY_CLIENT_SECRET, SPOTIFY_USER, SPOTIFY_REDIRECT_URI

## Authentification


In [2]:
# ID and password
cid = SPOTIFY_CLIENT_ID
secret = SPOTIFY_CLIENT_SECRET
username = SPOTIFY_USER
uri = SPOTIFY_REDIRECT_URI
scopes = 'user-read-private user-read-email'

In [3]:
# client_credentials_manager = SpotifyClientCredentials(client_id=cid, client_secret=secret)
# sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager)

In [9]:
sp = spotipy.Spotify(auth_manager=SpotifyClientCredentials(client_id="YOUR_APP_CLIENT_ID",
                                                           client_secret="YOUR_APP_CLIENT_SECRET"))

results = sp.search(q='bicep', limit=20)
for idx, track in enumerate(results['tracks']['items']):
    print(idx, track['name'])

0 Glue
1 Apricots
2 Apricots
3 Opal - Four Tet Remix
4 Atlas
5 Atlas
6 Siena
7 X
8 Lido
9 Saku
10 Aura
11 Cazenove
12 Bicep
13 In Yer Face - Bicep Remix
14 Light
15 Meli (I)
16 Rain
17 Saku
18 Sundial
19 Atlas


In [5]:
# sp = spotipy.Spotify(auth_manager=SpotifyOAuth(client_id=cid,
#                                                client_secret=secret,
#                                                redirect_uri=uri,
#                                                scope=scopes))

#results = sp.current_user_saved_tracks()
# for idx, item in enumerate(results['items']):
#     track = item['track']
#     print(idx, track['artists'][0]['name'], " – ", track['name'])

In [6]:
def spotify_authentification():
    """
    Api authentification using requests
    """
    AUTH_URL = 'https://accounts.spotify.com/api/token'

    # POST
    auth_response = requests.post(AUTH_URL, {
        'grant_type': 'client_credentials',
        'client_id': cid,
        'client_secret': secret,
    })

    # convert the response to JSON
    auth_response_data = auth_response.json()

    # save the access token
    access_token = auth_response_data['access_token']

    headers = {
        'Authorization': 'Bearer {token}'.format(token=access_token)
    }
    return headers

headers = spotify_authentification()

In [7]:
happiness = sp.playlist(playlist_id = '2oFSJfwxmLUqZUU4SgD0I7?si=671d061cbcfc4810')
sadness = sp.playlist(playlist_id = '0LbtLlb9G4M7TFi8OsDYMg?si=f6cf8536ee3c4e6f')
#neutral
worry = sp.playlist(playlist_id = '06Gh1LqdIfG1HMHKa44B7k?si=ad744fcfa57c4be3') #Same playlist as fear
love = sp.playlist(playlist_id = '37i9dQZF1DX7rOY2tZUw1k?si=88404786a9ae4cc6')
anger = sp.playlist(playlist_id = '5c5NfSIO6bMrUSoCZKMudz?si=211e93362660427b')
#surprise
fear = sp.playlist(playlist_id = '06Gh1LqdIfG1HMHKa44B7k?si=ad744fcfa57c4be3')#same playlist as worry
fun = sp.playlist(playlist_id = '7xOHp3ZlSBJNJOgsQwF85S?si=f04c656c9241412b')
relief = sp.playlist(playlist_id = '37i9dQZF1DWXe9gFZP0gtP?si=cefe9eafc2e14d8d')
hate = sp.playlist(playlist_id = '4OGYlQGUpMY6EZsmFdiJ1e?si=0252727c471f41e8')
enthusiasm = sp.playlist(playlist_id = '5t0hWEUHZ89a8jzBulvuvY?si=3dd98c1bb2c24bb2')
boredom = sp.playlist(playlist_id = '37i9dQZF1DWWQRwui0ExPn?si=575ab13eac8c418f')

ReadTimeout: HTTPSConnectionPool(host='api.spotify.com', port=443): Read timed out. (read timeout=5)

In [ ]:
feeling_playlists = [happiness, sadness, worry, love, anger, fear, fun, relief, hate, enthusiasm, boredom]

## Playlist function

In [ ]:
def select_and_play(feeling):
    for i in feeling_playlists:
        if i == feeling:
            return i.get('external_urls').values()

In [ ]:
select_and_play(hate)

In [ ]:
artist_album = sp.artist_albums(artist_id = '73A3bLnfnz5BoQjb4gNCga?si=U8ghJmknRGeBxCoUVn6NWQ')